# Movies Recommendations


### ETL

In [1]:
import pandas as pd
import numpy as np

Carga de datos

In [8]:
# Se extrae directamente desde un archivo en la carpeta de drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Se cargan los datos a un dataframe
movies = pd.read_csv("/content/drive/MyDrive/Modelo de Recomendaciones (Movies)/movies_dataset.csv")

<ipython-input-7-94e7192e2c80>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("/content/drive/MyDrive/Modelo de Recomendaciones (Movies)/movies_dataset.csv")


In [ ]:
# movies = pd.read_csv("/movies_dataset.csv")

In [95]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [24]:
movies_final = movies

In [42]:
movies_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45379 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45379 non-null  object 
 3   genres                 45379 non-null  object 
 4   homepage               7769 non-null   object 
 5   id                     45379 non-null  object 
 6   imdb_id                45365 non-null  object 
 7   original_language      45368 non-null  object 
 8   original_title         45379 non-null  object 
 9   overview               44438 non-null  object 
 10  popularity             45377 non-null  object 
 11  poster_path            45040 non-null  object 
 12  production_companies   45379 non-null  object 
 13  production_countries   45379 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Valores nulos de los campos revenue, budget rellenados por el número 0

In [35]:
movies_final['revenue'].fillna(0, inplace=True)

In [37]:
movies_final['budget'].fillna(0, inplace=True)

 Valores nulos del campo release_date  deben eliminarse

In [41]:
movies_final.dropna(subset=['release_date'], inplace=True)

Columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget

In [70]:
movies_final['return'] = movies_final['revenue'].astype(float) / movies_final['budget'].astype(float)

ValueError: ignored

In [64]:
# Mascara del valor erroneo
# mask = movies_final[(movies_final['revenue'] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg') | (movies_final['budget'] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg')]

In [66]:
# Borrar fila
# movies_final = movies_final.drop(mask.index)

Se observa un error en los datos de la columna 'budget' ya que deberia ser el presupuesto de la película, en dólares. Y hay datos tipo '/ ... .jpg'

Son varias columnas entonces:

In [71]:
# Convertir valores numéricos a float y mantener valores de texto como string
movies_final['budget'] = pd.to_numeric(movies_final['budget'], errors='coerce').fillna(movies_final['budget'])

In [73]:
# Errores
error = movies_final.loc[movies_final['budget'].apply(type) == str, 'budget']
error

29503    /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg
35587    /zaSf5OG7V8X8gqFvly88zDdRm46.jpg
Name: budget, dtype: object

In [74]:
# Borrar filas
movies_final = movies_final.drop(error.index)

In [85]:
movies_final['budget'] == 0

0        False
1        False
2         True
3        False
4         True
         ...  
45460     True
45462     True
45463     True
45464     True
45465     True
Name: budget, Length: 45376, dtype: bool

In [91]:
(movies_final['budget'] < 0).any()

False

In [96]:
# Calcular la columna "return" y asignar 0 cuando "budget" sea 0 ya que no se puede dividir entre 0
movies_final['return'] = np.where(movies_final['budget'] == 0, 0,
                                  movies_final['revenue'].astype(float) / movies_final['budget'].astype(float))

In [97]:
movies_final['return']

0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: return, Length: 45376, dtype: float64

Columnas que no serán utilizadas, video, imdb_id, adult, original_title, poster_path y homepage

In [98]:
Cols = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
movies_final = movies_final.drop(Cols, axis=1)

In [111]:
movies_final.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995


Fechas, con el formato AAAA-mm-dd

In [109]:
movies_final['release_date'] = pd.to_datetime(movies_final['release_date'])
movies_final['release_date'] = movies_final['release_date'].dt.strftime('%Y-%m-%d')


In [108]:
movies_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  object 
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  object 
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

 Crear la columna release_year donde se extraera el año de la fecha de estreno

In [110]:
movies_final['release_date'] = pd.to_datetime(movies_final['release_date'])
movies_final['release_year'] = movies_final['release_date'].dt.year

Desanidar Datos

In [118]:
dicc = [movies_final['belongs_to_collection']]
pd.DataFrame.from_dict(dicc)

,0,1,2,3,4,5,6,7,8,9,...,45455,45456,45457,45458,45459,45460,45462,45463,45464,45465
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,NaN,NaN,NaN,"{'id': 645, 'name': 'James Bond Collection', '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
credits = pd.read_csv("/content/drive/MyDrive/Modelo de Recomendaciones (Movies)/credits.csv")

In [13]:
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
